# NFL Baseline
- create target_df (distance in tracking_df is lower than threshold=3)
https://www.kaggle.com/code/stgkrtua/nfl-creatatraindataset-targetdf
- create dataset save frames in target_df
https://www.kaggle.com/code/stgkrtua/nfl-createdataset-saveframes
- check saved images
https://www.kaggle.com/code/stgkrtua/nfl-checkdataset-plotsavedimage

# import libraries

In [1]:
# general
import os
import gc
import pickle
import glob
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import cv2
import matplotlib.pyplot as plt
import time
import math

import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')
import timm


# deep learning
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts, ReduceLROnPlateau
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import StratifiedKFold

# loss metrics
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score

import mlflow
# import wandb
# warningの表示方法の設定
import warnings
warnings.filterwarnings("ignore")

# Set Configurations

In [2]:
CFG = {
        "kaggle" : False,
        "DEBUG" : False,
        # model config
        "model_name" : "tf_efficientnet_b5",
        "out_features" : 20,
        "inp_channels": 3*2,
        "num_extractFTR" : 5,
        "pretrained" : True,
        "features" : ['x_position_1', 'y_position_1', 'x_position_2', 'y_position_2', 
                      'speed_1', 'distance_1', 'direction_1', 'orientation_1','acceleration_1', 'sa_1', 
                      'speed_2', 'distance_2', 'direction_2', 'orientation_2', 'acceleration_2', 'sa_2',
                      'players_dis', 'is_ground'],
        "track_features_x_1" : ['x_position_shift-6_1','x_position_shift-5_1', 'x_position_shift-4_1',
                                'x_position_shift-3_1','x_position_shift-2_1', 'x_position_shift-1_1', 
                                'x_position_shift0_1','x_position_shift1_1', 'x_position_shift2_1', 
                                'x_position_shift3_1','x_position_shift4_1', 'x_position_shift5_1'],    
        "track_features_y_1" : ['y_position_shift-6_1','y_position_shift-5_1', 'y_position_shift-4_1',
                                'y_position_shift-3_1','y_position_shift-2_1', 'y_position_shift-1_1',
                                'y_position_shift0_1','y_position_shift1_1', 'y_position_shift2_1',
                                'y_position_shift3_1','y_position_shift4_1', 'y_position_shift5_1'],
        "track_features_x_2" : ['x_position_shift-6_2','x_position_shift-5_2', 'x_position_shift-4_2',
                                'x_position_shift-3_2','x_position_shift-2_2', 'x_position_shift-1_2',
                                'x_position_shift0_2','x_position_shift1_2', 'x_position_shift2_2',
                                'x_position_shift3_2','x_position_shift4_2', 'x_position_shift5_2'],
        "track_features_y_2" : ['y_position_shift-6_2','y_position_shift-5_2', 'y_position_shift-4_2',
                                'y_position_shift-3_2','y_position_shift-2_2', 'y_position_shift-1_2',
                                'y_position_shift0_2','y_position_shift1_2', 'y_position_shift2_2',
                                'y_position_shift3_2','y_position_shift4_2', 'y_position_shift5_2'],
        # learning config
        "n_epoch" : 7,
        "lr" : 1e-4,
        "T_max" : 10,
        "min_lr" : 1e-8,
        "weight_decay" : 1e-6,

        # etc
        "print_freq" : 1000,
        "random_seed" : 21,

        # data config    
        "img_size" : (224, 224),
        "batch_size" : 32,
        "num_workers" : 8,
        "masksize_helmet_ratio" : 4, # helmetサイズにこの係数をかけたサイズだけ色を残して後は黒塗りする
        "TRAIN_VIDEO_NUM" : 100,
        "VALID_VIDEO_NUM" : 10,
        "sample_num" : -1, 

        "EXP_CATEGORY" : "exps",
        "EXP_NAME" : "exp003_effnetb5_extractFTR",
}

if CFG["DEBUG"]:
    CFG["EXP_CATEGORY"] = "DEBUG"
    CFG["EXP_NAME"] = "DEBUG"
    CFG["n_epoch"] = 2
    CFG["sample_num"] = 1000
    CFG["batch_size"] = 32

if CFG["kaggle"]:
    CFG["INPUT_DIR"] = "/kaggle/input/"
    CFG["OUTPUT_DIR"] = "/kaggle/working/"
    CFG["TRAIN_HELMET_CSV"] = os.path.join(CFG["INPUT_DIR"], "nfl-player-contact-detection", "train_baseline_helmets.csv")
    CFG["TRAIN_TRACKING_CSV"] = os.path.join(CFG["INPUT_DIR"], "nfl-player-contact-detection", "train_player_tracking.csv")
    CFG["TRAIN_VIDEO_META_CSV"] = os.path.join(CFG["INPUT_DIR"], "nfl-player-contact-detection", "train_video_metadata.csv")
    CFG["TRAIN_LABEL_CSV"] = os.path.join(CFG["INPUT_DIR"], "nfl-player-contact-detection", "train_labels.csv")
    CFG["TARGET_CSV"] = os.path.join(CFG["INPUT_DIR"], "nfl-createdataset-saveframes-shift", "saved_frame_target.csv")
    CFG["TRAIN_E_IMG_DIR"] = os.path.join(CFG["INPUT_DIR"], "nfl-createdataset-saveframes-shift", "train_images")
    CFG["TRAIN_S_IMG_DIR"] = os.path.join(CFG["INPUT_DIR"], "nfl-createdataset-saveframes-shift-sview", "train_images")
    CFG["MODEL_DIR"] = CFG["OUTPUT_DIR"]
else:
    CFG["INPUT_DIR"] = "/workspace/input"
    CFG["OUTPUT_DIR"] = "/workspace/output"
    CFG["TRAIN_HELMET_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_baseline_helmets.csv")
    CFG["TRAIN_TRACKING_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_player_tracking.csv")
    CFG["TRAIN_VIDEO_META_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_video_metadata.csv")
    CFG["TRAIN_LABEL_CSV"] = os.path.join(CFG["INPUT_DIR"], "train_labels.csv")
    CFG["TARGET_CSV"] = os.path.join(CFG["INPUT_DIR"], "target_fillna0_shift_2.csv")
    CFG["TRAIN_E_IMG_DIR"] = os.path.join(CFG["INPUT_DIR"], "train_frames")
    CFG["TRAIN_S_IMG_DIR"] = CFG["TRAIN_E_IMG_DIR"]
    CFG["CONTACT_IMG_DIR"] = os.path.join(CFG["INPUT_DIR"], "contact_images")
    CFG["MODEL_DIR"] = os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"] ,"model")
    
if not CFG["kaggle"] and not CFG["DEBUG"]:
    os.mkdir(os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"]))
    os.mkdir(CFG["MODEL_DIR"])


In [3]:
if CFG["kaggle"]:
    os.environ["WANDB_SILENT"] = "true"
    WANDB_CONFIG = {'competition': 'NFL', '_wandb_kernel': 'taro'}
    # Secrets
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb")

    !wandb login $secret_value_0
    #! TODO : logger settings
    wandb.init(project=WANDB_CONFIG["competition"], config=CFG, group=CFG["EXP_CATEGORY"], name=CFG["EXP_NAME"])

else:
    mlflow.set_tracking_uri("/workspace/mlruns")
    experiment = mlflow.get_experiment_by_name(CFG["EXP_CATEGORY"])
    if experiment is None:  # 当該Experiment存在しないとき、新たに作成
        experiment_id = mlflow.create_experiment(name=CFG["EXP_CATEGORY"])
    else: # 当該Experiment存在するとき、IDを取得
        experiment_id = experiment.experiment_id

# Utils

In [4]:
def logging_metrics_epoch(fold, epoch, train_loss_avg, valid_loss_avg, score, threshold, tn_best, fp_best, fn_best, tp_best, auc_score):
    if CFG["kaggle"]:
        wandb.log({"loss avg":{f"train/fold{fold}": train_loss_avg,
                                f"valid/fold{fold}": valid_loss_avg}}, step=epoch)
        wandb.log({"Metircs" : {f"score/fold{fold}":score,
                                f"score threshold/fold{fold}":threshold,
                                f"tn/fold{fold}":tn_best,
                                f"fp/fold{fold}":fp_best,
                                f"fn/fold{fold}":fn_best,
                                f"tp/fold{fold}":tp_best,
                                f"auc/fold{fold}":auc_score,
                               }}, step=epoch)
    else:
        mlflow.log_metric(f"fold{fold} train loss avg", train_loss_avg, step=epoch)
        mlflow.log_metric(f"fold{fold} valid loss avg", valid_loss_avg, step=epoch)
        mlflow.log_metric(f"fold{fold} score", score, step=epoch)
        mlflow.log_metric(f"fold{fold} score threshold", threshold, step=epoch)
        mlflow.log_metric(f"fold{fold} tn", tn_best, step=epoch)
        mlflow.log_metric(f"fold{fold} fp", fp_best, step=epoch)
        mlflow.log_metric(f"fold{fold} fn", fn_best, step=epoch)
        mlflow.log_metric(f"fold{fold} tp", tp_best, step=epoch)
        mlflow.log_metric(f"fold{fold} auc", auc_score, step=epoch)

In [5]:
def seed_everything(seed=CFG["random_seed"]):
    #os.environ['PYTHONSEED'] = str(seed)
    np.random.seed(seed%(2**32-1))
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic =True
    torch.backends.cudnn.benchmark = False
seed_everything()

# device optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using device: {device}')

Using device: cuda


In [6]:
def asMinutes(s):
    """Convert Seconds to Minutes."""
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    """Accessing and Converting Time Data."""
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# Load Data

In [7]:
target_df = pd.read_csv(CFG["TARGET_CSV"])
# target_game_plays = target_df["game_play"].unique()[:50]
train_game_plays = target_df["game_play"].unique()[:CFG["TRAIN_VIDEO_NUM"]]
valid_game_plays = target_df["game_play"].unique()[-CFG["VALID_VIDEO_NUM"]:]
target_game_plays = list(set(train_game_plays) | set(valid_game_plays))
CFG["target_game_plays"] = list(target_game_plays)
target_df = target_df[target_df["game_play"].isin(target_game_plays)]

if CFG["DEBUG"]:
    target_df = target_df.sample(CFG["sample_num"])
elif CFG["sample_num"] != -1:
    target_df = target_df.sample(CFG["sample_num"])

target_df["is_ground"] = (target_df["nfl_player_id_2"] == "G").astype(np.int)
print(len(target_df))
display(target_df["contact"].value_counts())

305164


0    274357
1     30807
Name: contact, dtype: int64

In [8]:
# diff_cols = ['speed', 'distance', 'direction', 'orientation','acceleration', 'sa']

# for col in diff_cols:
#     target_df[f"{col}_diff"] = abs(target_df[f"{col}_1"] - target_df[f"{col}_2"])

In [9]:
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.3, 0.3), p=0.5),
    A.Normalize(mean=[0.], std=[1.]),
    ToTensorV2()
])

valid_transform = A.Compose([
    A.Normalize(mean=[0.], std=[1.]),
    ToTensorV2()
])

# Dataset

In [10]:
class NFLDataset(Dataset):
    def __init__(self, target_df, transform=None):
        self.target_df = target_df
        self.features = target_df[CFG["features"]].values
        self.track_features_x_1 = target_df[CFG["track_features_x_1"]].values
        self.track_features_y_1 = target_df[CFG["track_features_y_1"]].values
        self.track_features_x_2 = target_df[CFG["track_features_x_2"]].values
        self.track_features_y_2 = target_df[CFG["track_features_y_2"]].values
        self.transform = transform

    def __len__(self):
        return len(self.target_df)

    def __getitem__(self, idx):
        target_info = self.target_df.iloc[idx]
        features = self.features[idx]
        track_x_1 = self.track_features_x_1[idx]
        track_y_1 = self.track_features_y_1[idx]
        track_x_2 = self.track_features_x_2[idx]
        track_y_2 = self.track_features_y_2[idx]
        track_features = np.concatenate([track_x_1[np.newaxis, :],
                                         track_y_1[np.newaxis, :],
                                         track_x_2[np.newaxis, :],
                                         track_y_2[np.newaxis, :]])

        target = target_info.contact
        # read frame image
        game_play = target_info.game_play
        frame = target_info.frame
        contact_id = target_info.contact_id
        contact_fileid = f"{contact_id}_Endzone.jpg"
        contact_filename = os.path.join(CFG["CONTACT_IMG_DIR"], contact_fileid)
        img = cv2.imread(contact_filename)
        if img is None:
            img = np.zeros((224, 224, 3))
            img = np.transpose(img, (2, 0, 1)).astype(np.float)
            img = torch.tensor(img, dtype=torch.float)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if self.transform:
                img = self.transform(image=img)["image"]
            else:
                img = img / 255. # convert to 0-1
                img = np.transpose(img, (2, 0, 1)).astype(np.float32)
            img = torch.tensor(img, dtype=torch.float)
        input_img = img
        contact_fileid = f"{contact_id}_Sideline.jpg"
        contact_filename = os.path.join(CFG["CONTACT_IMG_DIR"], contact_fileid)
        img = cv2.imread(contact_filename)
        if img is None:
            img = np.zeros((224, 224, 3))
            img = np.transpose(img, (2, 0, 1)).astype(np.float)
            img = torch.tensor(img, dtype=torch.float)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if self.transform:
                img = self.transform(image=img)["image"]
            else:
                img = img / 255. # convert to 0-1
                img = np.transpose(img, (2, 0, 1)).astype(np.float32)
            img = torch.tensor(img, dtype=torch.float)
        input_img = np.concatenate([input_img, img], axis=0)
        input_img = torch.tensor(input_img, dtype=torch.float)
        target = torch.tensor(target, dtype=torch.float)
        features = torch.tensor(features, dtype=torch.float)
        track_features = torch.tensor(track_features, dtype=torch.float)
        return input_img, features, track_features, target

In [11]:
# checkするときは255.で割るのを全体にかける(normalizeがcheckにはないので真っ白とか真っ黒になるのを回避)
# show_img_num = 4

# check_transform = A.Compose([
#     A.HorizontalFlip(p=1.0),
#     A.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.3, 0.3), p=1.0),
# #     A.Normalize(mean=[0.], std=[1.]),
#     ToTensorV2()
# ])


# pick_df = target_df[target_df["contact"]==1]
# train_dataset = NFLDataset(pick_df, check_transform)
# train_loader = DataLoader(
#     train_dataset,
#     batch_size = show_img_num,
#     shuffle = False,
#     num_workers = CFG["num_workers"],
#     pin_memory = True
# )
# for batch_idx, (images, features, track_features, targets) in enumerate(train_loader):
#     fig = plt.figure(figsize=(12, 25))
#     print(images.shape)
#     print("features shape =",features.shape)
#     print("track features shape =",track_features.shape)
#     for idx in range(show_img_num):
#         img = images[idx].numpy()
#         img = img.transpose((1,2,0))
#         img = img[:, :, 0:3]
#         fig.add_subplot(1,show_img_num ,idx+1)
#         plt.imshow(img)
#         plt.title(targets[idx].numpy())
#     plt.show()
    
#     fig = plt.figure(figsize=(12, 25))
#     for idx in range(show_img_num):
#         img = images[idx].numpy()
#         img = img.transpose((1,2,0))
#         img = img[:, :, 3:6]
#         fig.add_subplot(1,show_img_num ,idx+1)
#         plt.imshow(img)
#         plt.title(targets[idx].numpy())
#     plt.show()
#     break
# del train_loader, train_dataset

# pick_df = target_df[target_df["contact"]==1]
# train_dataset = NFLDataset(pick_df)
# train_loader = DataLoader(
#     train_dataset,
#     batch_size = show_img_num,
#     shuffle = False,
#     num_workers = CFG["num_workers"],
#     pin_memory = True
# )
# for batch_idx, (images, features, track_features, targets) in enumerate(train_loader):
#     fig = plt.figure(figsize=(12, 25))
#     print(images.shape)
#     print("features shape =",features.shape)
#     print("track features shape =",track_features.shape)
#     for idx in range(show_img_num):
#         img = images[idx].numpy()
#         img = img.transpose((1,2,0))
#         img = img[:, :, 0:3]
#         fig.add_subplot(1,show_img_num ,idx+1)
#         plt.imshow(img)
#         plt.title(targets[idx].numpy())
#     plt.show()
    
#     fig = plt.figure(figsize=(12, 25))
#     for idx in range(show_img_num):
#         img = images[idx].numpy()
#         img = img.transpose((1,2,0))
#         img = img[:, :, 3:6]
#         fig.add_subplot(1,show_img_num ,idx+1)
#         plt.imshow(img)
#         plt.title(targets[idx].numpy())
#     plt.show()
#     break
# del train_loader, train_dataset

# Model

In [12]:
# without meta
class NFLNet(nn.Module):
    def __init__(
        self,
        model_name = CFG["model_name"],
        out_features = CFG["out_features"],
        inp_channels= CFG["inp_channels"],
        pretrained = CFG["pretrained"]
    ):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels, num_classes=out_features)
        self.mlp = nn.Sequential(
                        nn.Linear(len(CFG["features"]), 32),
                        nn.LayerNorm(32),
                        nn.ReLU(),
                        nn.Dropout(0.2),
                    )
        self.conv1 = nn.Sequential(
                        nn.Conv1d(4, 1, 5),
                        nn.Linear(len(CFG["track_features_y_2"])-4, 32),
                        nn.ReLU(),
                    )
        self.fc = nn.Linear(out_features+32+32, 1)
        self.emb = nn.Linear(out_features+32, CFG["num_extractFTR"])

    def forward(self, image, features, track_features):
        image_feature = self.model(image)
        features = self.mlp(features)
        track_features = self.conv1(track_features)
        if features.shape[0] == 1:
            track_featurs = track_featurs.reshape(1, -1)
        else:
            track_featurs = torch.squeeze(track_features)
        output = self.fc(torch.cat([image_feature, features, track_featurs], dim=1))
        embeddings = self.emb(torch.cat([image_feature, track_featurs], dim=1))
        return output, embeddings

# train fn

In [13]:
def train_fn(train_loader, model, criterion, epoch ,optimizer, scheduler):
    model.train()
    batch_time = AverageMeter()
    losses = AverageMeter()
    start, end = time.time(), time.time()
    for batch_idx, (images, features, track_features, targets) in enumerate(train_loader):
        images = images.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)      
        features = features.to(device, non_blocking = True).float()
        track_features = track_features.to(device, non_blocking = True).float()
        preds, _ = model(images, features, track_features)
        
        loss = criterion(preds, targets)
        losses.update(loss.item(), CFG["batch_size"]) 
        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        loss.backward() # パラメータの勾配を計算
        optimizer.step() # モデル更新
        optimizer.zero_grad() # 勾配の初期化
                
        batch_time.update(time.time() - end)
        end = time.time()
        if batch_idx % CFG["print_freq"] == 0 or batch_idx == (len(train_loader)-1):
            print('\t Epoch: [{0}][{1}/{2}] '
                    'Elapsed {remain:s} '
                    'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                    .format(
                        epoch, batch_idx, len(train_loader), batch_time=batch_time, loss=losses,
                        remain=timeSince(start, float(batch_idx+1)/len(train_loader)),
            ))
        del preds, images, features, targets
    gc.collect()
    torch.cuda.empty_cache()
    return losses.avg

# valid fn

In [14]:
def valid_fn(model, valid_loader, criterion):
    model.eval()# モデルを検証モードに設定
    test_targets = []
    test_preds = []

    batch_time = AverageMeter()
    losses = AverageMeter()
    start, end = time.time(), time.time()
    for batch_idx, (images, features, track_features, targets) in enumerate(valid_loader):
        images = images.to(device, non_blocking = True).float()
        targets = targets.to(device, non_blocking = True).float().view(-1, 1)
        features = features.to(device, non_blocking = True).float()
        track_features = track_features.to(device, non_blocking = True).float()
        with torch.no_grad():
            preds, _ = model(images, features, track_features)
            loss = criterion(preds, targets)
        losses.update(loss.item(), CFG["batch_size"])
        batch_time.update(time.time() - end)

        targets = targets.detach().cpu().numpy().ravel().tolist()
        preds = torch.sigmoid(preds).detach().cpu().numpy().ravel().tolist()

        test_preds.extend(preds)
        test_targets.extend(targets)
        # score = matthews_corrcoef(preds, targets)
        if batch_idx % CFG["print_freq"] == 0 or batch_idx == (len(valid_loader)-1):
            print('\t EVAL: [{0}/{1}] '
                'Elapsed {remain:s} '
                'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                .format(
                    batch_idx, len(valid_loader), batch_time=batch_time, loss=losses,
                    remain=timeSince(start, float(batch_idx+1)/len(valid_loader)),
                ))
        del preds, images, features, targets
        gc.collect()
        torch.cuda.empty_cache()
    test_preds = np.array(test_preds)
    test_targets = np.array(test_targets)
    return test_targets, test_preds, losses.avg

# Train loop

In [15]:
def training_loop(target_df):
    # set model & learning fn
    model = NFLNet()
    model = model.to(device)
    if CFG["kaggle"]:
        wandb.watch(model)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"], amsgrad=False)
    scheduler = CosineAnnealingLR(optimizer, T_max=CFG["T_max"], eta_min=CFG["min_lr"], last_epoch=-1)

    oof_df = pd.DataFrame()
    fold = 0
    # separate train/valid data 
    train_df = target_df[target_df["game_play"].isin(train_game_plays)]
    valid_df = target_df[target_df["game_play"].isin(valid_game_plays)]
    train_dataset = NFLDataset(train_df, train_transform)
    valid_dataset = NFLDataset(valid_df, valid_transform)
    train_loader = DataLoader(train_dataset,batch_size=CFG["batch_size"], shuffle = True,
                                num_workers = CFG["num_workers"], pin_memory = True)
    valid_loader = DataLoader(valid_dataset,batch_size=CFG["batch_size"], shuffle = False,
                                num_workers = CFG["num_workers"], pin_memory = True)

    # training
    best_score = -np.inf
    best_auc = -np.inf
    start_time, end = time.time(), time.time()
    for epoch in range(1, CFG["n_epoch"] + 1):
        print(f'\t === epoch: {epoch}: training ===')
        train_loss_avg = train_fn(train_loader, model, criterion, epoch ,optimizer, scheduler)
        valid_targets, valid_preds, valid_loss_avg = valid_fn(model, valid_loader, criterion)

        valid_score = -np.inf
        valid_threshold = 0
        tn_best, fp_best, fn_best, tp_best = 0, 0, 0, 0
        for idx in range(1, 10, 1):
            thr = idx*0.1
            valid_targets = (np.array(valid_targets) > thr).astype(np.int32)
            valid_binary_preds = (np.array(valid_preds) > thr).astype(np.int32)
            score_tmp = matthews_corrcoef(valid_targets, valid_binary_preds)
            cm = confusion_matrix(valid_targets, valid_binary_preds)
            tn, fp, fn, tp = cm.flatten()
            if score_tmp > valid_score:
                valid_score = score_tmp 
                valid_threshold = thr
                tn_best, fp_best, fn_best, tp_best = tn, fp, fn, tp
        elapsed = time.time() - start_time
        print(f'\t epoch:{epoch}, avg train loss:{train_loss_avg:.4f}, avg valid loss:{valid_loss_avg:.4f}, score:{valid_score:.4f}(th={valid_threshold}) ::: time:{elapsed:.2f}s')
        scheduler.step()
        # validationスコアがbestを更新したらモデルを保存する
        if valid_score > best_score:
            best_score = valid_score
            model_name = CFG["model_name"]
            torch.save(model.state_dict(), f'{CFG["MODEL_DIR"]}/{model_name}_fold{fold}.pth')
            print(f'\t Epoch {epoch} - Save Best Score: {best_score:.4f}. Model is saved.')
            contact_id = valid_df["contact_id"].values
            _oof_df = pd.DataFrame({
                "contact_id" : contact_id,
                "pred" : valid_preds,
                "contact" : valid_targets,
                "fold" : fold,
            })
            
        auc_score = roc_auc_score(valid_targets, valid_preds)
        if best_auc < auc_score:
            best_auc = auc_score
            model_name = CFG["model_name"]
            torch.save(model.state_dict(), f'{CFG["MODEL_DIR"]}/{model_name}_bestauc_fold{fold}.pth')
            print(f'\t Epoch {epoch} - Save Best AUC Score: {best_auc:.4f}. Model is saved.')
            
        logging_metrics_epoch(fold, epoch, train_loss_avg, valid_loss_avg, valid_score, valid_threshold, tn_best, fp_best, fn_best, tp_best, auc_score)

    del train_loader, train_dataset, valid_loader, valid_dataset
    oof_df = pd.concat([oof_df, _oof_df], axis = 0)
    del _oof_df
    gc.collect()
    torch.cuda.empty_cache()
    return oof_df

In [16]:
if CFG["kaggle"]:
    oof_df = training_loop(target_df)
    wandb.finish()
else:
    with mlflow.start_run(experiment_id=experiment_id, run_name=CFG["EXP_NAME"]) as run:
        mlflow.log_dict(CFG, "configuration.yaml")
        mlflow.log_param("positive data num", len(target_df[target_df["contact"]==1]))
        mlflow.log_param("negative data num", len(target_df[target_df["contact"]==0]))
        oof_df = training_loop(target_df)

	 === epoch: 1: training ===
	 Epoch: [1][0/8782] Elapsed 0m 2s (remain 322m 48s) Loss: 0.5600(0.5600) 
	 Epoch: [1][1000/8782] Elapsed 3m 47s (remain 29m 28s) Loss: 0.1233(0.2396) 
	 Epoch: [1][2000/8782] Elapsed 7m 34s (remain 25m 39s) Loss: 0.2215(0.2159) 
	 Epoch: [1][3000/8782] Elapsed 11m 19s (remain 21m 48s) Loss: 0.2406(0.2020) 
	 Epoch: [1][4000/8782] Elapsed 15m 5s (remain 18m 2s) Loss: 0.2563(0.1951) 
	 Epoch: [1][5000/8782] Elapsed 18m 54s (remain 14m 17s) Loss: 0.0553(0.1893) 
	 Epoch: [1][6000/8782] Elapsed 22m 39s (remain 10m 29s) Loss: 0.1238(0.1847) 
	 Epoch: [1][7000/8782] Elapsed 26m 24s (remain 6m 43s) Loss: 0.1114(0.1811) 
	 Epoch: [1][8000/8782] Elapsed 30m 10s (remain 2m 56s) Loss: 0.1665(0.1781) 
	 Epoch: [1][8781/8782] Elapsed 33m 5s (remain 0m 0s) Loss: 0.1519(0.1756) 
	 EVAL: [0/756] Elapsed 0m 1s (remain 14m 51s) Loss: 0.0299(0.0299) 
	 EVAL: [755/756] Elapsed 1m 44s (remain 0m 0s) Loss: 0.0692(0.1854) 
	 epoch:1, avg train loss:0.1756, avg valid loss:0.1854

In [17]:
display(oof_df)
if CFG["kaggle"]:
    oof_filename = os.path.join(CFG["OUTPUT_DIR"], "oof_df.csv")
    oof_df.to_csv(oof_filename, index=False)
else:
    oof_filename = os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"], "oof_df.csv")
    oof_df.to_csv(oof_filename, index=False)

,contact_id,pred,contact,fold
0,58573_000445_0_46131_47976,0.122918,0,0
1,58573_000445_0_47801_47976,0.008671,0,0
2,58573_000445_0_47801_52466,0.004526,0,0
3,58573_000445_0_44834_44888,0.085315,0,0
4,58573_000445_0_40171_47801,0.138245,0,0
...,...,...,...,...
24162,58582_003121_91_48220_G,0.030438,0,0
24163,58582_003121_91_47906_G,0.024292,0,0
24164,58582_003121_91_38557_G,0.038478,0,0
24165,58582_003121_91_47872_G,0.003342,0,0


# Create CNN-Emb df

In [18]:
target_df = pd.read_csv(CFG["TARGET_CSV"])
target_game_plays = target_df["game_play"].unique()[-100:]
CFG["target_game_plays"] = list(target_game_plays)
target_df = target_df[target_df["game_play"].isin(target_game_plays)]

if CFG["DEBUG"]:
    target_df = target_df.sample(CFG["sample_num"])
elif CFG["sample_num"] != -1:
    target_df = target_df.sample(CFG["sample_num"])

target_df = target_df.reset_index(drop=True)
target_df["is_ground"] = (target_df["nfl_player_id_2"] == "G").astype(np.int)
target_df["is_helmet"] = 1 - ((target_df["E_width_1"]==0) & (target_df["E_width_2"]==0) & (target_df["S_width_1"]==0) & (target_df["S_width_2"]==0)).astype(np.int)
print(len(target_df))
display(target_df["contact"].value_counts())

276406


0    251345
1     25061
Name: contact, dtype: int64

In [19]:
emb_dataset = NFLDataset(target_df, valid_transform)
emb_loader = DataLoader(
    emb_dataset,
    batch_size = CFG["batch_size"],
    shuffle = False,
    num_workers = CFG["num_workers"],
    pin_memory = True
)
model = NFLNet()
model = model.to(device)
# model.load_state_dict(torch.load(CFG["trained_model_path"]))
model.load_state_dict(torch.load(f'{CFG["MODEL_DIR"]}/{CFG["model_name"]}_fold0.pth'))
model.eval()# モデルを検証モードに設定
cnn_features = []
for batch_idx, (images, features, track_features, targets) in enumerate(emb_loader):
    print("\r {}/{}".format(batch_idx, len(emb_loader)), end="")
    images = images.to(device, non_blocking = True).float()
    features = features.to(device, non_blocking = True).float()
    track_features = track_features.to(device, non_blocking = True).float()    
    preds, emb = model(images, features, track_features)    
    emb_numpy = emb.detach().cpu().numpy()
    cnn_features.extend(emb_numpy)

 8637/8638

In [20]:
cnn_feature_df = pd.DataFrame(cnn_features, columns=[f'cnn_feature_{i}' for i in range(CFG["num_extractFTR"])])
display(cnn_feature_df)

,cnn_feature_0,cnn_feature_1,cnn_feature_2,cnn_feature_3,cnn_feature_4
0,3.263268,2.779468,-0.172263,0.761405,-0.693518
1,3.521633,5.494758,-1.474236,0.516962,1.030516
2,3.384871,6.329556,-1.425580,0.806048,1.474994
3,3.363161,7.447970,-2.209705,0.289219,2.755315
4,5.495241,8.164204,-2.804928,0.305526,3.879028
...,...,...,...,...,...
276401,3.680104,6.120726,-1.717427,1.541447,-1.495488
276402,6.990949,4.939717,0.164544,0.741985,-0.504377
276403,6.014412,4.948170,-0.775877,1.480881,-1.406000
276404,6.036087,7.086385,-0.804623,0.506808,-1.242071


In [21]:
concat_df = target_df.copy()
concat_df = pd.concat([target_df, cnn_feature_df], axis=1)
display(concat_df)
cnn_features_filename = os.path.join(CFG["OUTPUT_DIR"], CFG["EXP_NAME"] ,"target_cnn_emb.csv")
concat_df.to_csv(cnn_features_filename, index=False)

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact,frame,game_frame,game_frame_player_1,...,S_width_2,S_top_2,S_height_2,is_ground,is_helmet,cnn_feature_0,cnn_feature_1,cnn_feature_2,cnn_feature_3,cnn_feature_4
0,58403_001076_0_46082_46232,58403_001076,2020-12-27 21:47:57.800000+00:00,0,46082,46232,0,297,58403_001076_297,58403_001076_297_46082,...,17.0,302.0,18.0,0,1,3.263268,2.779468,-0.172263,0.761405,-0.693518
1,58403_001076_0_42424_46082,58403_001076,2020-12-27 21:47:57.800000+00:00,0,42424,46082,0,297,58403_001076_297,58403_001076_297_42424,...,20.0,346.0,26.0,0,1,3.521633,5.494758,-1.474236,0.516962,1.030516
2,58403_001076_0_42424_46232,58403_001076,2020-12-27 21:47:57.800000+00:00,0,42424,46232,0,297,58403_001076_297,58403_001076_297_42424,...,17.0,302.0,18.0,0,1,3.384871,6.329556,-1.425580,0.806048,1.474994
3,58403_001076_0_42424_44876,58403_001076,2020-12-27 21:47:57.800000+00:00,0,42424,44876,0,297,58403_001076_297,58403_001076_297_42424,...,19.0,274.0,26.0,0,1,3.363161,7.447970,-2.209705,0.289219,2.755315
4,58403_001076_0_42393_46649,58403_001076,2020-12-27 21:47:57.800000+00:00,0,42393,46649,0,297,58403_001076_297,58403_001076_297_42393,...,22.0,227.0,24.0,0,1,5.495241,8.164204,-2.804928,0.305526,3.879028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276401,58582_003121_91_48220_G,58582_003121,2021-10-12 02:42:29.100000+00:00,91,48220,G,0,840,58582_003121_840,58582_003121_840_48220,...,0.0,0.0,0.0,1,0,3.680104,6.120726,-1.717427,1.541447,-1.495488
276402,58582_003121_91_47906_G,58582_003121,2021-10-12 02:42:29.100000+00:00,91,47906,G,0,840,58582_003121_840,58582_003121_840_47906,...,0.0,0.0,0.0,1,0,6.990949,4.939717,0.164544,0.741985,-0.504377
276403,58582_003121_91_38557_G,58582_003121,2021-10-12 02:42:29.100000+00:00,91,38557,G,0,840,58582_003121_840,58582_003121_840_38557,...,0.0,0.0,0.0,1,0,6.014412,4.948170,-0.775877,1.480881,-1.406000
276404,58582_003121_91_47872_G,58582_003121,2021-10-12 02:42:29.100000+00:00,91,47872,G,0,840,58582_003121_840,58582_003121_840_47872,...,0.0,0.0,0.0,1,1,6.036087,7.086385,-0.804623,0.506808,-1.242071
